In [1]:
!pip install transformers -q
!pip install sentencepiece -q
!pip install -q -U tensorflow-text
!pip install -q tf-models-official

In [2]:
import os
import shutil

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization

import numpy as np
import pandas as pd

from transformers import AutoModel, AutoTokenizer 
from transformers import AdamW

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn as nn
device = torch.device("cuda")
cpu = torch.device("cpu")

In [3]:
data = pd.read_json('/content/drive/MyDrive/mp3_data_w_vectors.json')
#data = pd.read_json('https://raw.githubusercontent.com/pradeep-miriyala/multi-modal-bert-models/main/data/song_lyric_map.json?token=ADXRNFRS46PTRG46WUZLXHDBKH7HY')
data['iGenre'] = data.apply(lambda x:int(x.Genre=='Devotional'),axis=1)

In [4]:
indic_model = AutoModel.from_pretrained("ai4bharat/indic-bert")

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['predictions.decoder.bias', 'sop_classifier.classifier.weight', 'predictions.decoder.weight', 'sop_classifier.classifier.bias', 'predictions.dense.weight', 'predictions.LayerNorm.bias', 'predictions.dense.bias', 'predictions.bias', 'predictions.LayerNorm.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")

In [6]:
txt = list(data.apply(lambda x:x.Lyric,axis=1))

In [7]:
sent_id = tokenizer.batch_encode_plus(txt, padding=True, return_token_type_ids=False)

In [8]:
max_seq_len = 25
all_tokens = tokenizer.batch_encode_plus(txt, max_length=max_seq_len,padding='longest', truncation=True, return_token_type_ids=False)
all_seq = torch.tensor(all_tokens['input_ids'])
all_mask = torch.tensor(all_tokens['attention_mask'])
all_y = torch.tensor(data['iGenre'].tolist())

In [9]:
def get_data_loader(seq,mask,y,batch_size = 32):
  data = TensorDataset(seq, mask, y)
  sampler = RandomSampler(data)
  dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size)
  return (data,sampler,dataloader)

In [10]:
def get_data_loader(seq,mask,y,batch_size = 16):
  data = TensorDataset(seq, mask, y)
  sampler = RandomSampler(data)
  dataloader = DataLoader(data, sampler=sampler, batch_size=batch_size)
  return (data,sampler,dataloader)

In [11]:
for param in indic_model.parameters():
    param.requires_grad = True

In [12]:
class BERT_Arch(nn.Module):
    def __init__(self, bert):      
      super(BERT_Arch, self).__init__()
      self.bert = bert
      self.dropout = nn.Dropout(0.1)
      self.relu =  nn.ReLU()
      self.fc1 = nn.Linear(768,512)      
      self.fc2 = nn.Linear(512,2)
      self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, sent_id, mask):
      _, cls_hs = self.bert(sent_id, attention_mask=mask,return_dict=False)
      x = self.fc1(cls_hs)
      x = self.relu(x)
      x = self.dropout(x)
      x = self.fc2(x)
      x = self.softmax(x)
      return x

In [13]:
def train(model,train_dataloader,loss_fcn):
  model.train()
  total_loss, total_accuracy = 0, 0
  # empty list to save model predictions
  total_preds=[]  
  # iterate over batches
  for step,batch in enumerate(train_dataloader):    
    if step % 20 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))
    # push the batch to gpu
    batch = [r.to(device) for r in batch]
    sent_id, mask, labels = batch
    # clear previously calculated gradients 
    model.zero_grad()
    # get model predictions for the current batch
    preds = model(sent_id, mask)
    # compute the loss between actual and predicted values
    loss = loss_fcn(preds, labels)
    # add on to the total loss
    total_loss = total_loss + loss.item()
    # backward pass to calculate the gradients
    loss.backward()
    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    # update parameters
    optimizer.step()
    # model predictions are stored on GPU. So, push it to CPU
    preds=preds.detach().cpu().numpy()
    # append the model predictions
    total_preds.append(preds)
    del batch
  # compute the training loss of the epoch
  avg_loss = total_loss / len(train_dataloader)  
  # predictions are in the form of (no. of batches, size of batch, no. of classes).
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)
  #returns the loss and predictions
  return avg_loss, total_preds

In [14]:
def evaluate(model,val_dataloader,loss_fcn):
  print("\nEvaluating...")
  # deactivate dropout layers
  model.eval()
  total_loss, total_accuracy = 0, 0  
  # empty list to save the model predictions
  total_preds = []
  # iterate over batches
  for step,batch in enumerate(val_dataloader):    
    if step % 20 == 0 and not step == 0:      
      # Calculate elapsed time in minutes.
      elapsed = format_time(time.time() - t0)            
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))
    # push the batch to gpu
    batch = [t.to(device) for t in batch]
    sent_id, mask, labels = batch
    preds = model(sent_id, mask)
    # compute the validation loss between actual and predicted values
    loss = loss_fcn(preds,labels)
    total_loss = total_loss + loss.item()
    preds = preds.detach().cpu().numpy()
    total_preds.append(preds)
    del batch
  # compute the validation loss of the epoch
  avg_loss = total_loss / len(val_dataloader) 
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)
  return avg_loss, total_preds

In [15]:
k_folds = 5
# number of training epochs
epochs = 5
torch.manual_seed(42)
kfold = KFold(n_splits=k_folds, shuffle=True)
models = [BERT_Arch(indic_model) for x in range(k_folds)]
for fold, (train_ids, test_ids) in enumerate(kfold.split(data)):
  print(f'FOLD {fold}')
  print('--------------------------------')
  train_data, train_sampler, train_dataloader = get_data_loader(all_seq[train_ids],all_mask[train_ids],all_y[train_ids])
  test_data, test_sampler, test_dataloader = get_data_loader(all_seq[test_ids],all_mask[test_ids],all_y[test_ids])
  best_valid_loss = float('inf')
  models[fold].to(device)
  class_wts = compute_class_weight('balanced', np.unique(all_y[train_ids].tolist()), all_y[train_ids].tolist())
  print(class_wts)
  # convert class weights to tensor
  weights= torch.tensor(class_wts,dtype=torch.float)
  weights = weights.to(device)
  # loss function
  loss_fcn  = nn.NLLLoss(weight=weights)
  # empty lists to store training and validation loss of each epoch
  train_losses=[]
  valid_losses=[]
  # define the optimizer
  optimizer = AdamW(models[fold].parameters(), lr = 1e-5)
  #for each epoch
  for epoch in range(epochs):     
      print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
      #train model
      train_loss, _ = train(models[fold],train_dataloader,loss_fcn)    
      #evaluate model
      valid_loss, _ = evaluate(models[fold],test_dataloader,loss_fcn)    
      #save the best model
      if valid_loss < best_valid_loss:
          best_valid_loss = valid_loss
          torch.save(models[fold].state_dict(), 'saved_weights.pt')      
      # append training and validation loss
      train_losses.append(train_loss)
      valid_losses.append(valid_loss)    
      print(f'\nTraining Loss: {train_loss:.3f}')
      print(f'Validation Loss: {valid_loss:.3f}')
      torch.cuda.empty_cache()
  models[fold].load_state_dict(torch.load('saved_weights.pt'))
  preds = models[fold](all_seq[test_ids].to(device), all_mask[test_ids].to(device))
  preds = preds.detach().cpu().numpy()
  preds = np.argmax(preds, axis = 1)
  print('Test')
  print(classification_report(all_y[test_ids], preds))
  print(pd.crosstab(all_y[test_ids], preds))

FOLD 0
--------------------------------
[1.51730104 0.7457483 ]

 Epoch 1 / 5
  Batch    20  of     55.
  Batch    40  of     55.

Evaluating...

Training Loss: 0.693
Validation Loss: 0.695

 Epoch 2 / 5
  Batch    20  of     55.
  Batch    40  of     55.

Evaluating...

Training Loss: 0.691
Validation Loss: 0.689

 Epoch 3 / 5
  Batch    20  of     55.
  Batch    40  of     55.

Evaluating...

Training Loss: 0.683
Validation Loss: 0.681

 Epoch 4 / 5
  Batch    20  of     55.
  Batch    40  of     55.

Evaluating...

Training Loss: 0.640
Validation Loss: 0.633

 Epoch 5 / 5
  Batch    20  of     55.
  Batch    40  of     55.

Evaluating...

Training Loss: 0.593
Validation Loss: 0.626
Test
              precision    recall  f1-score   support

           0       0.52      0.79      0.62        81
           1       0.82      0.57      0.67       139

    accuracy                           0.65       220
   macro avg       0.67      0.68      0.65       220
weighted avg       0.71      